# Installation des packages

In [ ]:
import pandas as pd


# Préparation des données

In [ ]:
from scripts import data_collector as dc
from scripts import data_cleaner as dcl
landlocked_url = "https://en.wikipedia.org/wiki/List_of_countries_by_length_of_coastline"
codesISO_url = "https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes"

# Codes ISO-2 et ISO-3 relatifs aux pays

In [ ]:
codesISO_data = dc.get_ISOcodes(codesISO_url)
codesISO_data = dcl.clean_ISOData(codesISO_data)
pd.set_option('display.max_rows', None) # Pour afficher toutes les lignes
print(codesISO_data.head(200))

                                                  Pays ISO-2 ISO-3
0                                          Afghanistan    AF   AFG
1                                              Albania    AL   ALB
2                                              Algeria    DZ   DZA
3                                              Andorra    AD   AND
4                                               Angola    AO   AGO
5                                  Antigua and Barbuda    AG   ATG
6                                            Argentina    AR   ARG
7                                              Armenia    AM   ARM
8                                         Australia[f]    AU   AUS
9                                              Austria    AT   AUT
10                                          Azerbaijan    AZ   AZE
11                                       Bahamas (the)    BS   BHS
12                                             Bahrain    BH   BHR
13                                          Bangladesh    BD  

# PIB Réel

L'API permet de récupérer des données sur les pays via leurs codes ISO avec 2 lettres
Exemple : "FR" pour la France, "DE" pour l'Allemagne, "GB" pour le Royaume-Uni, etc.

In [ ]:
wb = dc.WorldBankData()

# Récupérer le PIB réel pour tous les pays du monde entre 2000 et 2024
liste_pays = codesISO_data["ISO-2"].tolist()
PIB_Reel_data = wb.get_indicator("PIB_reel", liste_pays, start=2015, end=2024)

PIB_Reel_data.head()

wb.plot("PIB_reel", title="Évolution du log(PIB réel) (2024 USD)")


ConnectionError: Erreur API : 403

# Imports Exports : 

In [ ]:
imports__data = wb.get_indicator("Importations", liste_pays, start=2015, end=2024)
exports_data = wb.get_indicator("Exportations", liste_pays, start=2015, end=2024)
imports_clean = imports__data.fillna(0)
exports_clean = exports_data.fillna(0)


# S'assurer que les deux DataFrames ont les mêmes colonnes
all_countries = set(imports_clean.columns).union(set(exports_clean.columns))

for country in all_countries:
    if country not in imports_clean.columns:
        imports_clean[country] = 0
    if country not in exports_clean.columns:
        exports_clean[country] = 0

# Calculer le solde commercial
balance_com = exports_clean - imports_clean
ratio = (exports_clean) / (imports_clean.replace(0, 1)) # Éviter la division par zéro

# Créer un DataFrame concaténé avec colonnes par pays
df_list = []
for country in sorted(all_countries):
    temp = pd.concat([
        imports_clean[[country]].rename(columns={country: f"{country}_import"}),
        exports_clean[[country]].rename(columns={country: f"{country}_export"}),
        balance_com[[country]].rename(columns={country: f"{country}_balance"}),
        ratio[[country]].rename(columns={country: f"{country}_ratio"})
    ], axis=1)
    df_list.append(temp)

# Concaténation finale : colonnes groupées par pays
df_combined = pd.concat(df_list, axis=1)
df_combined.index.name = "Year"


print(df_combined.head())



ConnectionError: Erreur API : 403

In [ ]:
from scripts import data_analyzer as da
TradeDataAnalyzer = da.TradeDataAnalyzer 

# Instancier
analyzer = TradeDataAnalyzer(imports_clean, exports_clean)

# 1. DataFrame combiné
df_combined = analyzer.get_combined_df()
print("Combined DF:")
print(df_combined.head())

# 2. Balance commerciale
balance_df = analyzer.get_balance()
print("\nBalance commerciale :")
print(balance_df.head())

# 3. Ratio export/import
ratio_df = analyzer.get_ratio()
print("\nRatio export/import :")
print(ratio_df.head())

# 4. Classification binaire
classification_df = analyzer.classify_countries()
print("\nClassification exportateur/importateur :")
print(classification_df.head())


ConnectionError: Erreur API : 403